In [2]:
! pip install svgwrite

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.1/67.1 kB 3.7 MB/s eta 0:00:00


In [3]:
import numpy as np
import svgwrite
from scipy.interpolate import splprep, splev

# Define a function to read CSV files containing polylines
def read_csv(csv_path):
    np_path_XYs = np.genfromtxt(csv_path, delimiter=',')
    path_XYs = []
    for i in np.unique(np_path_XYs[:, 0]):
        np_XYs = np_path_XYs[np_path_XYs[:, 0] == i][:, 1:]
        XYs = []
        for j in np.unique(np_XYs[:, 0]):
            XY = np_XYs[np_XYs[:, 0] == j][:, 1:]
            XYs.append(XY)
        path_XYs.append(XYs)
    return path_XYs

def is_circle(path):
    # Assume path is a 2D numpy array of shape (N, 2)
    center = np.mean(path, axis=0)
    radii = np.linalg.norm(path - center, axis=1)

    # Check if all radii are nearly equal (allowing a small tolerance)
    return np.allclose(radii, radii[0], rtol=1e-2, atol=1e-2)


def is_ellipse(path):
    center = np.mean(path, axis=0)
    distances = path - center

    # Calculate covariance matrix and eigenvalues (which correspond to the axes lengths)
    cov_matrix = np.cov(distances.T)
    eigenvalues, _ = np.linalg.eig(cov_matrix)

    # Check if there are two distinct eigenvalues (major and minor axes)
    # Allowing some tolerance to account for minor irregularities
    return np.isclose(eigenvalues[0], eigenvalues[1], rtol=1e-2, atol=1e-2) is False


def is_rectangle(path):
    if len(path) != 4:
        return False

    def angle_between(v1, v2):
        cos_theta = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
        return np.arccos(np.clip(cos_theta, -1.0, 1.0))

    # Check if opposite sides are equal and angles are approximately 90 degrees
    for i in range(4):
        v1 = path[(i + 1) % 4] - path[i]
        v2 = path[(i + 2) % 4] - path[(i + 1) % 4]

        if not np.isclose(np.linalg.norm(v1), np.linalg.norm(path[(i + 3) % 4] - path[(i + 2) % 4]), rtol=1e-2):
            return False

        if not np.isclose(angle_between(v1, v2), np.pi / 2, atol=1e-2):
            return False

    return True


def is_polygon(path):
    num_sides = len(path)

    if num_sides < 3:
        return False

    # Check if the path forms a closed loop
    if not np.allclose(path[0], path[-1]):
        return False

    # Calculate the internal angles and ensure consistency
    angles = []
    for i in range(1, num_sides - 1):
        v1 = path[i - 1] - path[i]
        v2 = path[i + 1] - path[i]
        angles.append(np.arctan2(np.linalg.det([v1, v2]), np.dot(v1, v2)))

    return np.allclose(angles, angles[0], rtol=1e-2, atol=1e-2)


# Identify regular shapes
def identify_shapes(paths_XYs):
    shapes = []
    for path in paths_XYs:
        if is_circle(path):
            shapes.append(('circle', path))
        elif is_ellipse(path):
            shapes.append(('ellipse', path))
        elif is_rectangle(path):
            shapes.append(('rectangle', path))
        elif is_polygon(path):
            shapes.append(('polygon', path))
        else:
            shapes.append(('other', path))
    return shapes

# Fit cubic Bézier curves
def fit_bezier_curves(path):
    path = np.array(path).reshape(-1, 2)
    tck, u = splprep([path[:, 0], path[:, 1]], s=0, per=True)
    new_points = splev(np.linspace(0, 1, 100), tck)
    return np.vstack(new_points).T

def convert_to_bezier(shapes):
    bezier_paths = []
    for shape, path in shapes:
        for segment in path:
            bezier_path = fit_bezier_curves(segment)
            bezier_paths.append(bezier_path)
    return bezier_paths

# Create an SVG file
def create_svg(bezier_paths, svg_path):
    dwg = svgwrite.Drawing(svg_path, profile='tiny')
    group = dwg.g()

    colours = ['red', 'green', 'blue', 'orange', 'purple', 'brown']

    for i, path in enumerate(bezier_paths):
        path_data = [("M", (path[0, 0], path[0, 1]))]
        for point in path[1:]:
            path_data.append(("L", (point[0], point[1])))
        path_data.append(("Z", None))
        group.add(dwg.path(d=path_data, fill='none', stroke=colours[i % len(colours)], stroke_width=2))

    dwg.add(group)
    dwg.save()

# Main function
def main():
    csv_path = 'frag2.csv'  # Replace with your CSV file path
    paths_XYs = read_csv(csv_path)
    shapes = identify_shapes(paths_XYs)
    bezier_paths = convert_to_bezier(shapes)
    create_svg(bezier_paths, 'output.svg')

if __name__ == "_main_":
    main()